**Installing the libraries**

In [ ]:
!pip install PyPDF2 langchain langchain_community langchain_google_genai google-generativeai transformers faiss-gpu

**Importing the libraries**

In [2]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from google.colab import userdata
google_api_key = userdata.get('google_api_key')

**Extracting the text from the pdf**

In [5]:
pdf_path = '/content/notes.pdf'
text = ""
with open(pdf_path, "rb") as file:
    pdf_reader = PdfReader(file)
    for page in pdf_reader.pages:
        text += page.extract_text()

**Splitting the text into chunks**

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
text_chunks = text_splitter.split_text(text)

**Converting the text chunks into embeddings**

In [7]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)
vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
vector_store.save_local("faiss_index")

**Creating a prompt template**

In [8]:
prompt_template = """
Answer the question in a detailed way and include all the related details, if the answer is not in
provided context just say, "Answer is not available", don't generate random responses\n\n
Context:\n {context}\n
Question: \n{question}\n

Answer:
"""

**Integrating the components using langchain**

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.4, google_api_key=google_api_key)
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

**Question Answering Inference**

In [13]:
question = 'What is the purpose of configuration items?'
db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs = db.similarity_search(question)

if not docs:
    print("No relevant documents found.")
else:
    response = chain.invoke({"input_documents": docs, "question": question})

    answer = response.get("output_text")
    print(answer)

Configuration items are used to identify, control, and manage change in software development projects. They are the items that are subject to change, such as programs, documentation, and data. By identifying and controlling these items, it is possible to ensure that changes are made in a controlled and orderly manner, and that the integrity of the software system is maintained.
